In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
sns.set()

In [ ]:
# вместо обычной LinearRegression будем использовать Ridge - более устойчивая модификация линейной регрессии
# для нормализации признаков используйте Ridge с параметром normalize=True
from sklearn.linear_model import Ridge
from sklearn.metrics import mean_squared_error

In [ ]:
# визуализация коэффициентов линейной регрессии
def visualize_coefficients(coefs, feature_names, top_n):
    """
    Функция для визуализации коэффициентов линейной регрессии.

    Параметры:
    -----------
        coefs: коэффициенты модели (model.coef_)
        feature_names: названия признаков (X_train.columns)
        top_n: вывести top_n самых положительных и top_n самых отрицательных признаков
    """
    feature_names = np.array(feature_names)
    if top_n * 2 > len(coefs):
        n_pos = len(coefs) // 2
        n_neg = len(coefs) - n_pos
    else:
        n_pos, n_neg = top_n, top_n
    # нам нужно найти индексы top_n наибольших и top_n наименьших коэффициентов
    min_coef_idxs = np.argsort(coefs)[:n_neg]
    max_coef_idxs = np.argsort(coefs)[len(coefs) - n_pos:]
    # соответствующие имена фичей
    top_feature_names = np.concatenate((feature_names[min_coef_idxs], feature_names[max_coef_idxs])) 
    # отобразим на bar-графике
    fig, ax = plt.subplots(figsize=(16, 9))
    ax.bar(np.arange(n_neg), coefs[min_coef_idxs], color=sns.xkcd_rgb['mauve'], hatch='/')
    ax.bar(np.arange(n_neg, n_neg + n_pos), coefs[max_coef_idxs], color=sns.xkcd_rgb['teal'], hatch='\\')
    ax.set_xticks(np.arange(0, n_neg + n_pos))
    ax.set_xticklabels(top_feature_names, rotation=45, ha="right", fontsize=14)
    plt.show()

In [ ]:
df = pd.read_csv('indian-metro.csv', parse_dates=['date_time'], index_col='date_time')

In [ ]:
df.head()

In [ ]:
# вы можете проверить, что признак dew_point - это копия признака visibility_in_miles
# поэтому его нужно удалить - линейные модели болеют от наличия абсолютно одинаковых признаков в датасете
df = df.drop('dew_point', axis=1)

In [ ]:
# выделим последний год в тестовую выборку, и еще один год в валидационную
test_start = df.index.max() - pd.Timedelta('1y')
val_start = test_start - pd.Timedelta('1y')

print(f'Test start date: {test_start}')
print(f'Validation start date: {val_start}')

In [ ]:
# в дальшейшем вы можете разбить свою выборку, выполнив
# train = df[df.index < val_start]
# val = df[(df.index >= val_start) & (df.index < test_start)]
# test = df[df.index >= test_start]

In [ ]:
def split_data(df, val_start, test_start):
    train = df[df.index < val_start]
    val = df[(df.index >= val_start) & (df.index < test_start)]
    test = df[df.index >= test_start]

    const_cols = train.columns[train.nunique() == 1]
    train = train.drop(const_cols, axis=1)
    val = val.drop(const_cols, axis=1)
    test = test.drop(const_cols, axis=1)
    return train, val, test

In [ ]:
def train_ridge(train, val):
    y_train = train['traffic_volume']
    X_train = train.drop('traffic_volume', axis=1)

    y_val = val['traffic_volume']
    X_val = val.drop('traffic_volume', axis=1)

    model = Ridge(normalize=True)
    model.fit(X_train, y_train)
    y_pred = model.predict(X_val)

    mse = mean_squared_error(y_val, y_pred)
    return model, mse, y_pred, X_train.columns

In [ ]:
def visualize_preds(y_true, y_pred, n_hours=336):
    fig, ax = plt.subplots(figsize=(21, 9))
    index = y_true[-n_hours:].index
    ax.plot(index, y_true[-n_hours:], label='y_true')
    ax.plot(index, y_pred[-n_hours:], label='y_pred')
    ax.legend()
    plt.show()

## Elementary level: 35%

*Задание*

*1. Удалите все нечисловные признаки из датасета*

*2. Выделите из выборки трейн и валидацию, не забудьте о масштабировании (нормализации) признаков*

*3. Обучите линейную регрессию на трейне, посчитайте MSE на валидации*

*4. Визуализируйте коэффициенты и предсказания. Для визуализации прогнозов возьмите, например, две последние недели в валидации*

*Замечание: после того, как вы разделите выборку может оказаться так, что в трейне некоторые признаки принимают константное значение. Их нужно удалить*

*Подсказка: для проверки количества уникальных значений в столбцах можно использовать df.nunique().sort_values()*

In [ ]:
def level1_preprocessing(df):
    df = df.copy()
    df = df.drop(['is_holiday', 'weather_type', 'weather_description'], axis=1)
    return df

In [ ]:
level1_df = level1_preprocessing(df)
level1_df.head()

In [ ]:
train, val, _ = split_data(level1_df, val_start, test_start)
model, val_mse, val_preds, feature_names = train_ridge(train, val)
print(f'Validation RMSE: {np.sqrt(val_mse)}')

In [ ]:
visualize_coefficients(model.coef_, feature_names, 15)

In [ ]:
visualize_preds(val['traffic_volume'], val_preds)

## Pre-intermediate: 25%

*Если вы построите график целевой переменной, то увидите ее сильную зависимость от даты*

*Создайте новые признаки года, месяца, дня и часа с помощью df.index.year, df.index.month и.т.п и проверьте качество на валидации*

*Не забудьте визуализировать предсказания и коэффициенты*

In [ ]:
def level2_preprocessing(df):
    df = df.copy()
    df = level1_preprocessing(df)
    df['year'] = df.index.year
    df['month'] = df.index.month
    df['day'] = df.index.day
    df['hour'] = df.index.hour
    return df

In [ ]:
level2_df = level2_preprocessing(df.copy())
train, val, _ = split_data(level2_df, val_start, test_start)
model, val_mse, val_preds, feature_names = train_ridge(train, val)
print(f'Validation RMSE: {np.sqrt(val_mse)}')

In [ ]:
visualize_coefficients(model.coef_, feature_names, 15)

In [ ]:
visualize_preds(val['traffic_volume'], val_preds)

## Intermediate: 20%

*Попробуйте закодировать отброшенные ранее категориальные признаки с помощью pd.get_dummies*

*Посмотрите на скор на валидации, визуализируйте результаты*

*Замечание: применять pd.get_dummies нужно до разделения на трейн/вал/тест, чтобы функция видела все значения категорий*

*Подсказка: к колонке weather_description лучше сначала применить .str.lower() - подумайте почему :)*

In [ ]:
def level3_preprocessing(df):
    df = df.copy()
    level2_df = level2_preprocessing(df)

    df['weather_description'] = df['weather_description'].str.lower()
    dummies = pd.get_dummies(df[['is_holiday', 'weather_type', 'weather_description']])
    return pd.concat([level2_df, dummies], axis=1)

In [ ]:
level3_df = level3_preprocessing(df)
level3_df.head()

In [ ]:
train, val, _ = split_data(level3_df, val_start, test_start)
model, val_mse, val_preds, feature_names = train_ridge(train, val)
print(f'Validation RMSE: {np.sqrt(val_mse)}')

In [ ]:
visualize_coefficients(model.coef_, feature_names, 15)

In [ ]:
visualize_preds(val['traffic_volume'], val_preds)

## Upper-intermediate: 20%

*На семинаре мы обсудили, что предыдущее значение целевой переменной может сильно помочь в предсказании временного ряда*

*Добавьте с помощью метода .shift() к колонкам сколько-то предыдущих значений целевой переменной, сколько конкретно - подберите на валидации*

*Визуализируйте прогнозы и коэффициенты модели*

In [ ]:
def add_lags(df, n_lags):
    df = df.copy()
    for lag_idx in range(1, n_lags + 1):
        df[f'lag_{lag_idx}'] = df['traffic_volume'].shift(lag_idx)
    return df

In [ ]:
from tqdm.auto import tqdm

lag_mses = []
for n_lags in tqdm(range(400)):
    lagged_df = add_lags(level3_df, n_lags=n_lags)
    train, val, _ = split_data(lagged_df, val_start, test_start)
    train = train.dropna()
    _, val_mse, _, _ = train_ridge(train, val)
    lag_mses.append(np.sqrt(val_mse))

In [ ]:
fig, ax = plt.subplots(figsize=(16, 9))
ax.plot(range(400), lag_mses)
ax.set_title(f'Best val_rsme = {min(lag_mses)} at {np.argmin(lag_mses)} lags')
plt.show()

In [ ]:
# будем использовать 8 лагов
def level4_preprocessing(df):
    df = df.copy()
    df = level3_preprocessing(df)
    df = add_lags(df, n_lags=8)
    return df

In [ ]:
level4_df = level4_preprocessing(df)
train, val, _ = split_data(level4_df, val_start, test_start)
train = train.dropna()
model, val_mse, val_preds, feature_names = train_ridge(train, val)
print(f'Validation RMSE: {np.sqrt(val_mse)}')

In [ ]:
visualize_coefficients(model.coef_, feature_names, 15)

In [ ]:
visualize_preds(val['traffic_volume'], val_preds)

## Advanced

In [ ]:
visualize_preds(val['traffic_volume'], val_preds, n_hours=36)

In [ ]:
fig, ax = plt.subplots(figsize=(16, 9))
level4_df.groupby('hour')['traffic_volume'].mean().plot(kind='bar', ax=ax)
plt.show()

In [ ]:
def level5_preprocessing(df):
    df = df.copy()
    df = level4_preprocessing(df)
    df['is_night'] = df['hour'].apply(lambda x: 0 <= x <= 4)
    df['is_morning_peak'] = df['hour'].apply(lambda x: 6 <= x <= 8)
    df['is_evening_peak'] = df['hour'].apply(lambda x: 15 <= x <= 17)
    df['weekday'] = df.index.weekday
    df['weekend'] = df['weekday'].apply(lambda x: x in (5, 6))
    return df

In [ ]:
level5_df = level5_preprocessing(df)
train, val, _ = split_data(level5_df, val_start, test_start)
train = train.dropna()
model, val_mse, val_preds, feature_names = train_ridge(train, val)
print(f'Validation RMSE: {np.sqrt(val_mse)}')

In [ ]:
visualize_coefficients(model.coef_, feature_names, 15)

In [ ]:
visualize_preds(val['traffic_volume'], val_preds)

In [ ]:
visualize_preds(val['traffic_volume'], val_preds, n_hours=36)

## Mastery

In [ ]:
df['weather_type'].value_counts()

In [ ]:
def level6_preprocessing(df):
    df = df.copy()
    df = level5_preprocessing(df)
    df['cos_hour'] = np.cos(2 * np.pi * df['hour'] / 24)
    df['sin_hour'] = np.sin(2 * np.pi * df['hour'] / 24)
    
    df['cos_weekday'] = np.cos(2 * np.pi * df['weekday'] / 7)
    df['sin_weekday'] = np.sin(2 * np.pi * df['weekday'] / 7)
    
    df['cos_month'] = np.cos(2 * np.pi * df['month'] / 12)
    df['sin_month'] = np.sin(2 * np.pi * df['month'] / 12)
    
    df['cos_wind_direction'] = np.cos(2 * np.pi * df['wind_direction'] / 360)
    df['sin_wind_direction'] = np.sin(2 * np.pi * df['wind_direction'] / 360)
    
    df['is_holiday'] = ~df['is_holiday_None'].astype('bool')
    df = df.drop('is_holiday_None', axis=1)
    df['not_bad_weather'] = df['weather_type_Clear'] | df['weather_type_Clouds']
    df['bad_weather'] = df['weather_type_Rain'] | df['weather_type_Snow'] | df['weather_type_Thunderstorm'] | df['weather_type_Squall']
    df['doubtful_weather'] = df['weather_type_Mist'] | df['weather_type_Drizzle'] | df['weather_type_Haze'] | df['weather_type_Fog']
    return df

In [ ]:
level6_df = level6_preprocessing(df)
train, val, _ = split_data(level6_df, val_start, test_start)
train = train.dropna()
model, val_mse, val_preds, feature_names = train_ridge(train, val)
print(f'Validation RMSE: {np.sqrt(val_mse)}')

In [ ]:
visualize_coefficients(model.coef_, feature_names, 20)

In [ ]:
visualize_preds(val['traffic_volume'], val_preds)

In [ ]:
visualize_preds(val['traffic_volume'], val_preds, n_hours=36)

In [ ]:
train, val, test = split_data(level6_df, val_start, test_start)
train = train.dropna()
train = pd.concat([train, val], axis=0)
model, test_mse, test_preds, feature_names = train_ridge(train, test)
print(f'Test RMSE: {np.sqrt(test_mse)}')

In [ ]:
visualize_preds(test['traffic_volume'], test_preds)